In [270]:
import pandas as pd
import numpy as np

In [271]:
import csv

In [293]:
df = pd.read_csv('/Users/anush/veg2vec/veg_latin.txt') 

In [294]:
def add_pf(x):
    if len(x) > 0:
        rng = np.arange(1, len(x)).astype('str').astype('object')
        pf = np.concatenate([[''], '-' + rng])
        x['vernacular'] = x.vernacular + pf
    return x

In [295]:
df = df.groupby(['vernacular', 'type']).apply(add_pf)

In [296]:
df

,vernacular,latin,type
0,amla,phyllanthaceae,family
1,amla,phyllanthus,genus
2,amla,phyllanthus_emblica,species
3,basil,lamiaceae,family
4,basil,ocimum,genus
5,basil,ocimum_basilicum,species
6,broccoli,brassicaceae,family
7,broccoli,brassica,genus
8,broccoli,brassica_oleracea,species
9,carrot,apiaceae,family


In [297]:
df2 = df.pivot(index='vernacular', columns='type', values='latin')

In [298]:
df3 = df2.dropna()

In [299]:
df3

type,family,genus,species
vernacular,,,
amla,phyllanthaceae,phyllanthus,phyllanthus_emblica
basil,lamiaceae,ocimum,ocimum_basilicum
broccoli,brassicaceae,brassica,brassica_oleracea
carrot,apiaceae,daucus,daucus_carota
chamomile,asteraceae,matricaria,matricaria_chamomilla
cumin,apiaceae,cuminum,cuminum_cyminum
echinacea,asteraceae,echinacea,echinacea_purpurea
fenugreek,fabaceae,trigonella,trigonella_foenum_graecum
garlic,amaryllidaceae,allium,allium_sativum


In [300]:
df4 = df3.reset_index().set_index('species')

In [301]:
df4

type,vernacular,family,genus
species,,,
phyllanthus_emblica,amla,phyllanthaceae,phyllanthus
ocimum_basilicum,basil,lamiaceae,ocimum
brassica_oleracea,broccoli,brassicaceae,brassica
daucus_carota,carrot,apiaceae,daucus
matricaria_chamomilla,chamomile,asteraceae,matricaria
cuminum_cyminum,cumin,apiaceae,cuminum
echinacea_purpurea,echinacea,asteraceae,echinacea
trigonella_foenum_graecum,fenugreek,fabaceae,trigonella
allium_sativum,garlic,amaryllidaceae,allium


In [302]:
def add_higher_types(x):
    n = x.name.split('-')[0]
    try:
        x.family = df3.loc[n].family
        x.genus = df3.loc[n].genus
    except KeyError:
        x.family = df4.loc[x.species].family
        x.genus = df4.loc[x.species].genus
    return x

In [303]:
df_final = (
    df2.apply(add_higher_types, axis=1).reset_index()
    .assign(vernacular=df_final.vernacular.str.split('-').str[0])
)

In [304]:
import itertools

In [305]:
column_pairs =list(map(list, itertools.combinations(df_final.columns, 2)))

In [306]:
column_pairs

[['vernacular', 'family'],
 ['vernacular', 'genus'],
 ['vernacular', 'species'],
 ['family', 'genus'],
 ['family', 'species'],
 ['genus', 'species']]

In [307]:
def make_pairs(df, colms):
    pairs = map(list, df[colms].drop_duplicates().values)
    return list(itertools.permutations(pairs, 2))

In [308]:
analogies = {tuple(p): make_pairs(df_final, p) for p in column_pairs}

In [309]:
with open('/Users/anush/veg2vec/analogies_veg.txt', 'w') as f:
    for ind, ((i, j), pairs) in enumerate(analogies.items(), 1):
        f.write(':{}-{}\n'.format(i, j))
        for ind2, (x, y) in enumerate(pairs, 1):
            if (ind == len(column_pairs)) and (ind2 == len(pairs)):
                end = ''
            else:
                end = '\n'
            f.write('{} {} {} {}{}'.format(*x, *y, end))

In [310]:
sum(map(len, analogies.values())) + len(analogies)

2072

In [311]:
phrases = [i for i in df_final.values.ravel() if '_' in i]

In [312]:
phrases

['phyllanthus_emblica',
 'ocimum_basilicum',
 'brassica_oleracea',
 'brussel_sprout',
 'brassica_oleracea',
 'brussel_sprouts',
 'brassica_oleracea',
 'daucus_carota',
 'brassica_oleracea',
 'matricaria_chamomilla',
 'brassica_oleracea',
 'brassica_oleracea',
 'cuminum_cyminum',
 'echinacea_purpurea',
 'trigonella_foenum_graecum',
 'allium_sativum',
 'zingiber_officinale',
 'brassica_oleracea',
 'brassica_oleracea',
 'citrus_limon',
 'long_pepper',
 'piper_longum',
 'citrus_×_sinensis',
 'citrus_sinensis',
 'ocimum_tenuiflorum',
 'curcuma_longa']

In [313]:
with open('/Users/anush/veg2vec/phrases.txt', 'w') as f:
    f.write('\n'.join(phrases))

In [315]:
values = np.concatenate([i.split('_') for i in df_final.values.ravel()])

In [319]:
with open('/Users/anush/veg2vec/samples_1/vocab.text') as f:
    words = f.read().split('\n')